<a href="https://colab.research.google.com/github/AkotoChristine/Online_Fraud_Detection/blob/main/Summative_Intro_to_ml_%5Bname_number%5D_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization Techniques in Machine Learning

Objective: This assignment aims to explore implementation or Machine Learning Models with regularization, optimization and Error analysis  techniques used in machine learning to improve models' performance, convergence speed, and efficiency..

A Notebook detailing the following

* Project name
* Clear out puts from cells






**Instructions**

1. Acquire a dataset suitable for ML tasks as per your proposal.
2. Implement a simple machine learning model based on neural networks on the chosen dataset without any defined optimization techniques. (Check instructions)
3. Implement and compare the model's performance after applying 3 to 4 disntict combinations regularization and optimization techniques.
4. Discuss the results on the README file.
5. Make predictions using test data
7. Implement error analysis techniques and ensure there is: F1-Score, Recall, Precision, RUC a confusion matrix using plotting libraries (not verbose)

Submit notebook to github repo




# Case Study and Implementation




In [ ]:
#Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# The Dataset
> ***Brief Description:***
State the Problem and A short Description of the data


In [ ]:
#TO DO: Load Data (Seprate into: Train, Validation and test sets)

In [16]:
data=pd.read_csv('onlinefraud.csv')  
data.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [ ]:
data.shape

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
data['isFraud'].value_counts(normalize=True)


In [ ]:
data['type'].nunique() #df['nameDest'].nunique()


In [ ]:
data['step'].nunique()


In [ ]:
numeric_df = data.select_dtypes(include="number")
plt.figure(figsize=(10, 6))
sns.heatmap(numeric_df, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
#dropping data['isFlaggedFraud'] since it wont be needed 
data = data.drop('isFlaggedFraud', axis =1)

In [ ]:
(data['oldbalanceOrg'] == data["oldbalanceDest"]).value_counts()

In [ ]:
(data['newbalanceOrig'] == data["oldbalanceOrg"]).value_counts()

## data preprocessing 

In [ ]:
data['type'] = data['type'].map({'PAYMENT':0, 'CASH_IN':1, 'DEBIT':2, 'CASH_OUT':3, 'TRANSFER':4})

#SECTION 1: Model Architecture:



```
TODO: Insert an image with the Model architecture here.Replace the image Below
```
> <img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*v1ohAG82xmU6WGsG2hoE8g.png" alt="?" style="width:25px"/>




#Task: Define a function that creates models without and With specified Optimization techniques


In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizer
from tensorflow.keras.callbacks import EarlyStopping
pip install -r requirements.txt

def define_model(optimization: string, regularization_datatype, early_stopping: bool, dropout: float, learning_rate: float):
  model= None
  model.add(None)
  #TO DO: Add more layers as per architecture
  model.add(None) # Last Layer
  model.compile(optimizer = optimizerNone)
  model.fit(None)
  return model

# Task: Print out the Final Model Accuracy and plot the Loss curve

In [ ]:
def loss_curve_plot(None):
  epochs = None
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title('Training and Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

# SECTION 2: Optimization and Regularization Combinations
At this point you should now create models that combine various optimization techniques
As done before make sure to plot out the loss curve and the accuracy and loss in verbose

In [ ]:
#TODO:
model_2 = define_model('Adam', None)
loss_curve_plot(model_2):
#print out confusion matrix and error analysis metrics after the cell

In [ ]:
#TODO:
model_3 = define_model('RMSPop',None)
loss_curve_plot(model_3):
#print out confusion matrix and error analysis metrics after the cell

In [ ]:
#TODO:
model_4 = define_model(None)
loss_curve_plot(model_4):
#print out confusion matrix and error analysis metrics after the cell

#Task: Make Predictions using the best saved model


Create a confusion Matrix and F1 score for both Models. Ensure outputs for the cells are visible

Finally, Make predictions using the best model. By the time you get to this cell you may realise at some point you needed to save the model so that you cal load it later

In [ ]:
def make_predictions(model_path, X):

    # Load the model
    model = load_model(None)
    # Make predictions
    predictions = None
    # Convert probabilities to binary labels (0 or 1)

    return predictions

#Modify the code appropriately

In [ ]:
model_path = None
make_predictions(None)

Congratulations!!
